# Création du jeu de données d'évaluation

In [18]:
import os
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

La génération de question d'évaluation se divise en deux groupes : 
- Les questions "temporelles" qui impliquent alors une année spécifique
- Les questions "généralles" qui n'ont pas de temporalité associée

Pour ces deux groupes, nous avons 5 catégories de questions :
- Questions temporelles :
    - Données démographiques et socio-économiques
    - Projets, réalisation, urbanisme
    - Budget et finance
    - Évenements et manifestations
    - Gouvernance et ressources humaines
- Questions généralles :
    - Compétences et missions
    - Équipements et infrastructures
    - Géographie et territoire
    - Communication et outils numérique
    - Chiffres clés

Pour le groupe temporel, nous allons générer 2 questions par catégorie par année (2 questions par 5 catégories pour 16 rapports d'activités = 160 questions). Au moment de la génération, nous allons également envoyer le résultat brut de l'OCR pour que les questions générées soient adaptées. 

Pour le groupe général, nous allons générer 40 questions par catégories. Nous fournirons également les 110 questions générées par le groupe temporel pour que le LLM connaisse notre contexte et adapte ses questions.

### Groupe temporel

#### Création des questions

In [63]:
prompt = """
Ton objectif est de générer des questions sur un rapport d'activité d'une année précise d'une communauté de commune.

Tu devras générer 2 questions par catégorie de questions qui sont les suivantes avec deux exemple : 
- Données démographiques et socio-économiques : Quelle était le nombre d'habitant en 2008 ? Combien d’emplois ont été créés sur le territoire en 2008 ?
- Projets, réalisation, urbanisme et infrastructures : Quels projets ont été inaugurés en 2008 par la CAVF ? Quels équipements culturels ont ouvert en 2008 ?
- Budget et finance : Quel était le montant des principales dépenses d’investissement de la CAVF en 2008 ? Quel était le coût total de la réhabilitation de la piscine de Nilvange en 2008 ?
- Évenements et manifestations : Quels événements culturels ont été organisés au parc du haut-fourneau U4 en 2008 ? Quels étaient les temps forts de la saison culturelle 2008 ?
- Gouvernance et ressources humaines : Quels étaient les membres du bureau de la CAVF en 2008 ? Quels étaient les rôles du conseil de développement en 2008 ?

Commme tu peux le constater, tu peux potentiellement adapter tes questions aux données spécifiques du rapport que tu as.

Répond selon au format JSON suivant les deux questions demandées par catégorie :

```json
"questions" : {
    "demographie": [
        "question 1",
        "question 2",
    ],
    "projets": [
        "question 1",
        "question 2",
    ],
    "finance": [
        "question 1",
        "question 2",
    ],
    "evenements": [
        "question 1",
        "question 2",
    ],
    "gouvernance": [
        "question 1",
        "question 2",
    ],
}
```

Et voici le document : 

<document>
DOCUMENT_TO_REPLACE


</document>

Le document est terminé.


Respectes bien le format JSON attendu avec les mêmes noms clés de catégories, et n'oublies pas de rédiger des questions pour l'année YEAR_TO_REPLACE qui peuvent être répondues avec le document que tu reçois, étant de cette année YEAR_TO_REPLACE
"""

In [ ]:
import json
from collections import defaultdict

answers = defaultdict()
with open("../datasets/ocr_raw.json") as file:
    data = json.load(file)

    for year in range(2009, 2025):
        raw_ocr = "\n".join([line[1] for line in data[str(year)]])

        answer = client.chat.complete(
            model="mistral-small-2506",
            messages=[
                {
                    "role": "user",
                    "content": prompt.replace("DOCUMENT_TO_REPLACE", raw_ocr).replace("YEAR_TO_REPLACE", str(year)),
                }
                ],
                response_format={
                    "type": "json_object",
                }
        )
        
        answers[str(year)] = json.loads(answer.choices[0].message.content)["questions"]


In [68]:
rows = []

for year in range(2009, 2025):
    for category, questions in answers[str(year)].items():
        rows.append({
            "question": questions[0],
            "category": category,
            "year": year
        })
        rows.append({
            "question": questions[1],
            "category": category,
            "year": year
        })

#### Récupération des documents pertinents

### Groupe général

### Création et sauvegarde du dataset